# 05 - H2 Dissociation

**Overview** 

This notebook guides you through ...  

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.special import erf
from time import perf_counter
!pip install -q pyscf > /dev/null
from pyscf import gto, scf, ao2mo, fci

**Problem** 

Lorem Ipsum ... 

**Model**

Lorem Ipsum ...

>Smart question?

Lorem Ipsum ...

## Part 1: Understanding Quantum Chemistry Basis Sets

**Questions**

Before you run any simulation, answer the following question(s):

1. What are the main differences between a gaussian and the solution of the hydrogen atom (exponential decay)?
2. If you need to represent the electron-electron interaction in a basis set of atomic functions centered on the different atoms, how many integrals do you need to compute? 

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Other questions

In [ ]:
# @title GTOs vs STOs { display-mode: "form" }
n = 1 # @param {type:"integer"}
n_gto = 6 # @param {type:"integer"}
z = 1 # @param {type:"integer"}
r_max = 10.0 # @param {type:"number"}
logscale = False # @param {type:"boolean"}

# ==== Radial functions & inner products ====
def R_sto(n, z, r):
    return r**(n - 1) * np.exp(-z * r)

def R_gto_component(n, alpha, r):
    return r**(n - 1) * np.exp(-alpha * r**2)

def gto_sum(n, alphas, coeffs, r):
    total = np.zeros_like(r)
    for a, c in zip(alphas, coeffs):
        total += c * R_gto_component(n, a, r)
    return total

def ip(f, g, r):
    return np.trapz(4.0 * np.pi * r**2 * f * g, r)

def norm(f, r):
    val = ip(f, f, r)
    return np.sqrt(val) if val > 0 else 0.0

def normalized_overlap(f, g, r):
    nf, ng = norm(f, r), norm(g, r)
    if nf == 0 or ng == 0:
        return 0.0
    return ip(f, g, r) / (nf * ng)

# Given alphas, compute optimal linear coefficients by solving S c = b
# where S_ij = <g_i|g_j>, b_i = <g_i|f>.
def solve_coeffs(n, alphas, f, r, reg=1e-12):
    G = [R_gto_component(n, a, r) for a in alphas]
    m = len(alphas)
    S = np.empty((m, m))
    for i in range(m):
        for j in range(i, m):
            Sij = ip(G[i], G[j], r)
            S[i, j] = Sij
            S[j, i] = Sij
    # Gentle Tikhonov to avoid ill-conditioning at high n_gto
    S = S + reg * np.eye(m)
    b = np.array([ip(g, f, r) for g in G])
    try:
        c = np.linalg.solve(S, b)
    except np.linalg.LinAlgError:
        c = np.linalg.lstsq(S, b, rcond=None)[0]
    return c

# ==== Grid ====
r = np.linspace(1e-9, r_max, 4000)  # finer grid improves integrals
f_target = R_sto(n, z, r)

# ==== Objective: only alphas are free; coeffs solved in closed form ====
def obj_from_logalphas(x):
    alphas = np.exp(x)
    coeffs = solve_coeffs(n, alphas, f_target, r)
    fit = gto_sum(n, alphas, coeffs, r)
    return -normalized_overlap(f_target, fit, r)  # minimize negative overlap

# ==== Multi-start for robustness ====
rng = np.random.default_rng(0)
alpha_lo, alpha_hi = 0.03 * z**2, 25.0 * z**2
log_lo, log_hi = np.log(alpha_lo), np.log(alpha_hi)

starts = []
# 1) deterministic log-spaced seed
starts.append(np.linspace(log_lo, log_hi, n_gto))
# 2) a few jittered variants around the log-spaced seed
for _ in range(5):
    base = np.linspace(log_lo, log_hi, n_gto)
    jitter = rng.normal(0.0, 0.4, size=n_gto)  # moderate jitter in log space
    starts.append(base + jitter)
# 3) one random strictly sorted sample
rnd = np.sort(rng.uniform(log_lo, log_hi, size=n_gto))
starts.append(rnd)

best = None
for x0 in starts:
    res = minimize(obj_from_logalphas, x0, method="L-BFGS-B",
                   options=dict(maxiter=500, ftol=1e-12))
    if (best is None) or (res.fun < best.fun):
        best = res

log_alphas_opt = best.x
alphas_opt = np.exp(log_alphas_opt)
coeffs_opt = solve_coeffs(n, alphas_opt, f_target, r)
fit = gto_sum(n, alphas_opt, coeffs_opt, r)
ov = normalized_overlap(f_target, fit, r)

# ==== Probability densities ====
def radial_prob(psi_r, r):
    return 4.0 * np.pi * r**2 * np.abs(psi_r)**2

P_sto = radial_prob(f_target, r)
P_fit = radial_prob(fit, r)

# ==== Plot ====
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.58, 0.42],
    subplot_titles=(
        f"Radial Functions (n={n}, z={z}, n_gto={n_gto}) — overlap={ov:.6f}",
        "Radial Probability Density  $4\\pi r^2 |\\phi(r)|^2$"
    ),
    horizontal_spacing=0.1
)

# Left panel: STO + fit
fig.add_trace(go.Scatter(x=r, y=f_target, name='STO', mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=r, y=fit, name='GTO sum (fit)', mode='lines'), row=1, col=1)

# Individual Gaussians, but exclude them from legend
dash_cycle = ['dash', 'dot', 'dashdot', 'longdash', 'longdashdot']
for i, (a, c) in enumerate(zip(alphas_opt, coeffs_opt)):
    comp = c * R_gto_component(n, a, r)
    fig.add_trace(
        go.Scatter(
            x=r, y=comp, mode='lines',
            showlegend=False,  # <--- suppress from legend
            line=dict(dash=dash_cycle[i % len(dash_cycle)], width=1.2, color='gray')
        ),
        row=1, col=1
    )

fig.update_yaxes(type='log' if logscale else 'linear', row=1, col=1)
fig.update_xaxes(title_text='r (bohr)', row=1, col=1)
fig.update_yaxes(title_text='Radial Function', row=1, col=1)

# Right panel: probability densities
fig.add_trace(go.Scatter(x=r, y=P_sto, name='P_STO', mode='lines'), row=1, col=2)
fig.add_trace(go.Scatter(x=r, y=P_fit, name='P_GTO sum (fit)', mode='lines'), row=1, col=2)
fig.update_xaxes(title_text='r (bohr)', row=1, col=2)
fig.update_yaxes(title_text='Probability Density', row=1, col=2)

# Legend inside the left plot
fig.update_layout(
    title='STO vs GTO Fit (linear coeffs, max-overlap objective)',
    legend=dict(
        orientation='v',
        x=0.3, y=0.95,   # adjust position inside left subplot
        xanchor='left', yanchor='top',
        bgcolor='rgba(255,255,255,0.6)',  # semi-transparent background
        bordercolor='black', borderwidth=1
    ),
    margin=dict(l=30, r=30, t=70, b=50)
)

fig.show()

In [ ]:
# @title Why Gaussians? (Overlap / e–n / e–e) { display-mode: "form" }
alpha = 1.0 # @param {type:"number"}
beta = 1.0  # @param {type:"number"}
gamma = 1.0 # @param {type:"number"}
delta = 1.0 # @param {type:"number"}
distance = 1.0 # @param {type:"number"}
n_gto = 6   # @param {type:"integer"}
integral = "electron-electron"  # @param ["overlap", "electron-nuclear", "electron-electron"]
sto_mc_samples = 100_000_000  # Monte Carlo samples for STO (per integral)
sto_mc_repeats = 2          # repeat MC to stabilize timing
rng_seed = 0

# ------------------------------------------------------------
# Normalized 1s STO and s-GTO
# ------------------------------------------------------------
def sto_1s(zeta, r):
    return (zeta**3/np.pi)**0.5 * np.exp(-zeta * r)

def gto_1s(expnt, r):
    return (2*expnt/np.pi)**0.75 * np.exp(-expnt * r**2)

def ip_radial(f, g, r):
    return np.trapz(4*np.pi * r**2 * f * g, r)

# ------------------------------------------------------------
# Fit STO(zeta) ≈ Σ_i c_i gto_1s(a_i) at same center
#   - optimize a_i (in log-space); solve c_i from normal equations
# ------------------------------------------------------------
def _solve_coeffs(zeta, alphas, r, reg=1e-12):
    f = sto_1s(zeta, r)
    G = [gto_1s(a, r) for a in alphas]
    m = len(alphas)
    S = np.empty((m, m))
    for i in range(m):
        for j in range(i, m):
            val = ip_radial(G[i], G[j], r)
            S[i, j] = S[j, i] = val
    S += reg * np.eye(m)
    b = np.array([ip_radial(g, f, r) for g in G])
    try:
        c = np.linalg.solve(S, b)
    except np.linalg.LinAlgError:
        c = np.linalg.lstsq(S, b, rcond=None)[0]
    return c

def fit_sto_with_ngto(zeta, n_gto, r):
    f = sto_1s(zeta, r)
    def overlap_from_logalphas(x):
        alphas = np.exp(x)
        coeffs = _solve_coeffs(zeta, alphas, r)
        gsum = np.sum([c*gto_1s(a, r) for a, c in zip(alphas, coeffs)], axis=0)
        num = ip_radial(f, gsum, r)
        nf  = np.sqrt(ip_radial(f, f, r))
        ng  = np.sqrt(ip_radial(gsum, gsum, r))
        return num / (nf*ng) if (nf>0 and ng>0) else 0.0

    rng = np.random.default_rng(0)
    lo, hi = np.log(0.03*zeta**2), np.log(25.0*zeta**2)
    starts = [np.linspace(lo, hi, n_gto)]
    for _ in range(4):
        starts.append(np.linspace(lo, hi, n_gto) + rng.normal(0, 0.4, n_gto))
    starts.append(np.sort(rng.uniform(lo, hi, n_gto)))

    best = None
    for x0 in starts:
        res = minimize(lambda x: -overlap_from_logalphas(x), x0,
                       method="L-BFGS-B", options=dict(maxiter=400, ftol=1e-12))
        if (best is None) or (res.fun < best.fun):
            best = res

    alphas = np.exp(best.x)
    coeffs = _solve_coeffs(zeta, alphas, r)
    return alphas, coeffs

# ------------------------------------------------------------
# Boys F0(t) with safe small-t expansion
# ------------------------------------------------------------
def boys0(t):
    t = np.asarray(t)
    out = np.empty_like(t, dtype=float)
    small = t < 1e-8
    ts = t[small]
    out[small] = 1.0 - ts/3.0 + ts**2/10.0 - ts**3/42.0  # series
    tt = t[~small]
    out[~small] = 0.5*np.sqrt(np.pi)/np.sqrt(tt) * erf(np.sqrt(tt))
    return out

# ------------------------------------------------------------
# Analytic (normalized) primitive integrals for s-type GTOs
# All formulas assume unnormalized Gaussians exp(-a|r-A|^2); we multiply by
# normalization constants N_a = (2a/pi)^(3/4) etc to convert to normalized.
# ------------------------------------------------------------
def _Na(a): return (2*a/np.pi)**0.75

def overlap_pair_ss_norm(a, b, A, B):
    p = a + b
    mu = a*b/p
    R2 = np.dot(A-B, A-B)
    # unnormalized overlap: (pi/p)^(3/2) * exp(-mu R^2)
    S0 = (np.pi/p)**1.5 * np.exp(-mu*R2)
    return _Na(a)*_Na(b) * S0

def nuc_attract_pair_ss_norm(a, b, A, B, C):  # ⟨a|1/|r-C||b⟩
    p = a + b
    mu = a*b/p
    R2 = np.dot(A-B, A-B)
    P = (a*A + b*B)/p
    RPC2 = np.dot(P-C, P-C)
    # unnormalized: -2*pi/p * exp(-mu R^2) * F0(p |P-C|^2); drop minus sign (return magnitude)
    V0 = 2*np.pi/p * np.exp(-mu*R2) * boys0(p*RPC2)
    return _Na(a)*_Na(b) * V0

def eri_pair_ssss_norm(a, b, c, d, A, B, C, D):  # (ab|cd)
    p = a + b
    q = c + d
    mu = a*b/p
    nu = c*d/q
    RAB2 = np.dot(A-B, A-B)
    RCD2 = np.dot(C-D, C-D)
    P = (a*A + b*B)/p
    Q = (c*C + d*D)/q
    RPQ2 = np.dot(P-Q, P-Q)
    # unnormalized (ss|ss):
    # 2*pi^(2.5) / (p*q*sqrt(p+q)) * exp(-mu RAB^2 - nu RCD^2) * F0( (p*q/(p+q)) * |P-Q|^2 )
    pref = 2 * np.pi**2.5 / (p*q*np.sqrt(p+q))
    T = (p*q/(p+q)) * RPQ2
    ERI0 = pref * np.exp(-mu*RAB2 - nu*RCD2) * boys0(T)
    return _Na(a)*_Na(b)*_Na(c)*_Na(d) * ERI0

# ------------------------------------------------------------
# Contracted analytic integrals by summing primitive pairs
# (coeffs are for normalized primitives)
# ------------------------------------------------------------
def contracted_overlap_norm(alA, cA, alB, cB, A, B):
    S = 0.0
    for a, ca in zip(alA, cA):
        for b, cb in zip(alB, cB):
            S += ca*cb * overlap_pair_ss_norm(a, b, A, B)
    # normalize to unit-norm contracted functions
    SAA = contracted_overlap_raw(alA, cA, alA, cA, A, A)
    SBB = contracted_overlap_raw(alB, cB, alB, cB, B, B)
    return S / np.sqrt(SAA * SBB)

def contracted_overlap_raw(alA, cA, alB, cB, A, B):
    S = 0.0
    for a, ca in zip(alA, cA):
        for b, cb in zip(alB, cB):
            S += ca*cb * overlap_pair_ss_norm(a, b, A, B)
    return S

def contracted_nuclear_norm(alA, cA, alB, cB, A, B, C):
    V = 0.0
    for a, ca in zip(alA, cA):
        for b, cb in zip(alB, cB):
            V += ca*cb * nuc_attract_pair_ss_norm(a, b, A, B, C)
    # norm by ||A||·||B||
    SAA = contracted_overlap_raw(alA, cA, alA, cA, A, A)
    SBB = contracted_overlap_raw(alB, cB, alB, cB, B, B)
    return V / np.sqrt(SAA * SBB)

def contracted_eri_norm(alA, cA, alB, cB, alC, cC, alD, cD, A, B, C, D):
    V = 0.0
    for a, ca in zip(alA, cA):
        for b, cb in zip(alB, cB):
            for c, cc in zip(alC, cC):
                for d, cd in zip(alD, cD):
                    V += ca*cb*cc*cd * eri_pair_ssss_norm(a, b, c, d, A, B, C, D)
    # normalize by ||AB|| and ||CD|| piecewise: here use product of single-function norms
    SAA = contracted_overlap_raw(alA, cA, alA, cA, A, A)
    SBB = contracted_overlap_raw(alB, cB, alB, cB, B, B)
    SCC = contracted_overlap_raw(alC, cC, alC, cC, C, C)
    SDD = contracted_overlap_raw(alD, cD, alD, cD, D, D)
    return V / np.sqrt(SAA*SBB*SCC*SDD)

# ------------------------------------------------------------
# STO Monte Carlo estimators
# ------------------------------------------------------------
def sto_value_at_point(zeta, center, xyz):
    r = np.linalg.norm(xyz - center, axis=-1)
    return (zeta**3/np.pi)**0.5 * np.exp(-zeta * r)

def mc_overlap_sto(zA, zB, A, B, n_samples, rng):
    # sample in a sphere that encloses both centers
    R = np.linalg.norm(A-B)
    tail = 6.0 / max(min(zA, zB), 1e-8)
    radius = max(8.0, tail + 0.5*R)
    vol = (4.0/3.0) * np.pi * radius**3
    u = rng.random(n_samples)
    cos_t = rng.uniform(-1.0, 1.0, size=n_samples)
    phi = rng.uniform(0.0, 2*np.pi, size=n_samples)
    r = radius * u**(1/3)
    sin_t = np.sqrt(1.0 - cos_t**2)
    xyz = np.stack([r*sin_t*np.cos(phi), r*sin_t*np.sin(phi), r*cos_t], axis=1)
    f = sto_value_at_point(zA, A, xyz) * sto_value_at_point(zB, B, xyz)
    est = vol * f.mean()
    err = vol * f.std(ddof=1) / np.sqrt(n_samples)
    return est, err

def mc_en_sto(zA, zB, A, B, C, n_samples, rng):
    # ⟨A| 1/|r-C| |B⟩
    RAB = np.linalg.norm(A-B)
    tail = 6.0 / max(min(zA, zB), 1e-8)
    radA = tail + 0.5*RAB
    radC = max(radA, np.linalg.norm(C-(A+B)/2)+tail)
    radius = max(8.0, radC)
    vol = (4.0/3.0) * np.pi * radius**3
    u = rng.random(n_samples)
    cos_t = rng.uniform(-1.0, 1.0, size=n_samples)
    phi = rng.uniform(0.0, 2*np.pi, size=n_samples)
    r = radius * u**(1/3)
    sin_t = np.sqrt(1.0 - cos_t**2)
    xyz = np.stack([r*sin_t*np.cos(phi), r*sin_t*np.sin(phi), r*cos_t], axis=1)
    denom = np.linalg.norm(xyz - C, axis=1) + 1e-12
    f = sto_value_at_point(zA, A, xyz) * sto_value_at_point(zB, B, xyz) / denom
    est = vol * f.mean()
    err = vol * f.std(ddof=1) / np.sqrt(n_samples)
    return est, err

def mc_ee_sto(zA, zB, zC, zD, A, B, C, D, n_pairs, rng):
    # crude 6D MC: sample r1 in sphere around AB, r2 in sphere around CD independently
    def sample_sphere(radius, n):
        u = rng.random(n)
        cos_t = rng.uniform(-1.0, 1.0, size=n)
        phi = rng.uniform(0.0, 2*np.pi, size=n)
        r = radius * u**(1/3)
        sin_t = np.sqrt(1.0 - cos_t**2)
        return np.stack([r*sin_t*np.cos(phi), r*sin_t*np.sin(phi), r*cos_t], axis=1)

    RAB = np.linalg.norm(A-B); RCD = np.linalg.norm(C-D)
    tail1 = 6.0 / max(min(zA, zB), 1e-8)
    tail2 = 6.0 / max(min(zC, zD), 1e-8)
    rad1 = max(8.0, tail1 + 0.5*RAB)
    rad2 = max(8.0, tail2 + 0.5*RCD)
    V1 = (4.0/3.0) * np.pi * rad1**3
    V2 = (4.0/3.0) * np.pi * rad2**3

    r1 = sample_sphere(rad1, n_pairs)
    r2 = sample_sphere(rad2, n_pairs)

    psiA = sto_value_at_point(zA, A, r1)
    psiB = sto_value_at_point(zB, B, r1)
    psiC = sto_value_at_point(zC, C, r2)
    psiD = sto_value_at_point(zD, D, r2)
    denom = np.linalg.norm(r1 - r2, axis=1) + 1e-12
    f = psiA*psiB*psiC*psiD / denom
    est = V1*V2 * f.mean()
    err = V1*V2 * f.std(ddof=1) / np.sqrt(n_pairs)
    return est, err

# ------------------------------------------------------------
# Build radial grid; fit all four STOs at their own centers (fits are single-center)
# ------------------------------------------------------------
R = distance
A = np.array([-R/2, 0.0, 0.0])   # center for α
B = np.array([+R/2, 0.0, 0.0])   # center for β
C = A.copy()                      # center for γ  (same as α)
D = B.copy()                      # center for δ  (same as β)
# If you want a distinct nuclear center for e–n, place it here:
C_nuc = np.array([0.0, 0.0, 0.0])  # nucleus at origin (adjust as desired)

r_max = max(20.0/min(alpha, beta, gamma, delta), 15.0)
r = np.linspace(1e-8, r_max, 5000)

alA, cA = fit_sto_with_ngto(alpha, n_gto, r)
alB, cB = fit_sto_with_ngto(beta,  n_gto, r)
alC, cC = fit_sto_with_ngto(gamma, n_gto, r)
alD, cD = fit_sto_with_ngto(delta,  n_gto, r)

# ------------------------------------------------------------
# Compute + time: STO (MC) vs contracted GTO (analytic)
# ------------------------------------------------------------
rng = np.random.default_rng(rng_seed)

if integral == "overlap":
    # STO (MC)
    sto_vals, sto_errs, sto_times = [], [], []
    for _ in range(sto_mc_repeats):
        t0 = perf_counter()
        val, err = mc_overlap_sto(alpha, beta, A, B, sto_mc_samples, rng)
        t1 = perf_counter()
        sto_vals.append(val); sto_errs.append(err); sto_times.append(t1 - t0)
    sto_val = float(np.mean(sto_vals)); sto_err = float(np.mean(sto_errs)); sto_time = float(np.median(sto_times))
    # cGTO (analytic, normalized)
    t0 = perf_counter()
    cgt_val = contracted_overlap_norm(alA, cA, alB, cB, A, B)
    t1 = perf_counter()
    cgt_time = t1 - t0
    print(f"=== OVERLAP  (R = {R:.3f} bohr) ===")
    print(f"STO–STO (MC):         {sto_val:.8f}  (± {sto_err:.2e})   time ≈ {sto_time*1e3:.1f} ms")
    print(f"cGTO–cGTO (analytic): {cgt_val:.8f}                     time ≈ {cgt_time*1e3:.3f} ms")

elif integral == "electron-nuclear":
    # STO (MC)
    sto_vals, sto_errs, sto_times = [], [], []
    for _ in range(sto_mc_repeats):
        t0 = perf_counter()
        val, err = mc_en_sto(alpha, beta, A, B, C_nuc, sto_mc_samples, rng)
        t1 = perf_counter()
        sto_vals.append(val); sto_errs.append(err); sto_times.append(t1 - t0)
    sto_val = float(np.mean(sto_vals)); sto_err = float(np.mean(sto_errs)); sto_time = float(np.median(sto_times))
    # cGTO (analytic, normalized)
    t0 = perf_counter()
    cgt_val = contracted_nuclear_norm(alA, cA, alB, cB, A, B, C_nuc)
    t1 = perf_counter()
    cgt_time = t1 - t0
    print(f"=== ELECTRON–NUCLEAR  ⟨α|1/|r-C||β⟩  (R = {R:.3f} bohr, C at {C_nuc}) ===")
    print(f"STO (MC):             {sto_val:.8f}  (± {sto_err:.2e})   time ≈ {sto_time*1e3:.1f} ms")
    print(f"cGTO (analytic):      {cgt_val:.8f}                     time ≈ {cgt_time*1e3:.3f} ms")

elif integral == "electron-electron":
    # STO (MC) — 6D; use fewer pairs if needed for speed
    ee_pairs = max(100_000, sto_mc_samples // 10)  # heuristic to keep runtime reasonable
    t0 = perf_counter()
    sto_val, sto_err = mc_ee_sto(alpha, beta, gamma, delta, A, B, C, D, ee_pairs, rng)
    t1 = perf_counter()
    sto_time = t1 - t0
    # cGTO (analytic, normalized)
    t0 = perf_counter()
    cgt_val = contracted_eri_norm(alA, cA, alB, cB, alC, cC, alD, cD, A, B, C, D)
    t1 = perf_counter()
    cgt_time = t1 - t0
    print(f"=== ELECTRON–ELECTRON  (αβ|γδ)  (R = {R:.3f} bohr) ===")
    print(f"STO (MC 6D):          {sto_val:.8f}  (± {sto_err:.2e})   time ≈ {sto_time:.2f} s")
    print(f"cGTO (analytic):      {cgt_val:.8f}                     time ≈ {cgt_time*1e3:.3f} ms")
else:
    raise ValueError("Unknown 'integral' option.")


## Part 2: Hartree-Fock Calculations of H2 Dissociation

In [ ]:
# @title Running PySCF for H2 (max HF detail) { display-mode: "form" }
distance = 1.0 # @param {type:"number"}
basis_set = "STO-3G"  # @param ["STO-3G", "4-31G", "6-31G*", "6-311G**", "cc-pVDZ", "cc-pVTZ"]

L = distance/2 + 3.0  # box half-length (bohr)

# ---------- Build H2 (bond along z here to match your original slice) ----------
R = float(distance)
mol = gto.Mole()
mol.atom  = f'H 0 0 {-R/2}; H 0 0 {R/2}'
mol.unit  = 'Bohr'
mol.basis = basis_set
mol.spin  = 0  # singlet
mol.charge= 0
mol.build()

print(mol)  # AO/basis summary

# ---------- RHF with very verbose logging ----------
mf = scf.RHF(mol)
mf.max_cycle = 100
mf.conv_tol  = 1e-12
mf.diis_space = 12
mf.verbose = 5         # 0=silent … 4=INFO … 5=DEBUG (prints iterations, DIIS, timings)
ehf = mf.kernel()      # SCF log will print to output

print("\n=== Post-SCF summary ===")
print(f"Converged: {mf.converged}")
print(f"RHF Energy (E_tot): {ehf:.12f} Ha")
print(f"Nuclear repulsion (E_nuc): {mol.energy_nuc():.12f} Ha")

# 1-RDM and core / Fock pieces
dm1   = mf.make_rdm1()
hcore = mf.get_hcore()
Sovlp = mol.intor('int1e_ovlp')
Tkin  = mol.intor('int1e_kin')
Vnuc  = mol.intor('int1e_nuc')

E_core = float(np.einsum('ij,ji->', hcore, dm1))          # <H_core> = <T + V_nuc>
E_kin  = float(np.einsum('ij,ji->', Tkin, dm1))           # <T>
E_enuc = float(np.einsum('ij,ji->', Vnuc, dm1))           # <V_nuc>
E_2e   = ehf - mol.energy_nuc() - E_core                  # Coulomb+exchange contribution

print(f"<T>               : {E_kin:.12f} Ha")
print(f"<V_nuc>           : {E_enuc:.12f} Ha")
print(f"<H_core>=<T+Vnuc> : {E_core:.12f} Ha")
print(f"Two-electron part : {E_2e:.12f} Ha")

# Orbital spectrum / occupations
orb_E = mf.mo_energy
occ   = mf.mo_occ
nelec_a = mol.nelectron // 2
homo = nelec_a - 1
lumo = nelec_a if nelec_a < orb_E.size else None
gap  = (orb_E[lumo] - orb_E[homo]) if lumo is not None else np.nan

print("\nOrbital energies (Ha) and occupations:")
for i, (e, o) in enumerate(zip(orb_E, occ)):
    tag = []
    if i == homo: tag.append("HOMO")
    if lumo is not None and i == lumo: tag.append("LUMO")
    tag_s = ("  [" + ", ".join(tag) + "]") if tag else ""
    print(f"  MO {i:2d}:  e = {e: .8f}   occ = {o:.1f}{tag_s}")
if lumo is not None:
    print(f"HOMO–LUMO gap: {gap:.8f} Ha  ({gap*27.211386:.3f} eV)")

# Mulliken population / AO labels
print("\nMulliken population analysis:")
mf.mulliken_pop(mol, dm1, s=Sovlp)   # prints table with AO labels & charges

# Overlap matrix eigenvalues (condition check)
evals = np.linalg.eigvalsh(Sovlp)
print("\nOverlap matrix S eigenvalues (min..max): "
      f"{evals.min():.6e} .. {evals.max():.6e}")

# (Optional) print small matrices for STO-3G size
nao = mol.nao_nr()
if nao <= 10:
    np.set_printoptions(precision=6, suppress=True)
    print("\nS (overlap):\n", Sovlp)
    print("\nH_core (T+Vnuc):\n", hcore)
    print("\nDensity matrix P (α+β):\n", dm1)

# ---------- Tiny FCI for reference ----------
m  = mf.mo_coeff.shape[1]
h1 = mf.mo_coeff.T @ mf.get_hcore() @ mf.mo_coeff
h2 = ao2mo.restore(1, ao2mo.kernel(mol, mf.mo_coeff), m)
cisolver = fci.FCI(mol, mf.mo_coeff)
cisolver.verbose = 4
efci, fcivec = cisolver.kernel(h1, h2, m, mol.nelectron)
print(f"\nFCI Energy: {efci:.12f} Ha")

# ---------- Quick 2D MO slices (x–z plane, y=0) ----------
x = np.linspace(-L, L, 200)
z = np.linspace(-L, L, 200)
X, Z = np.meshgrid(x, z, indexing='xy')
coords = np.stack([X.ravel(), np.zeros_like(X.ravel()), Z.ravel()], axis=1)

ao_values = mol.eval_gto('GTOval_sph', coords)
mo_vals = ao_values @ mf.mo_coeff
mo1 = mo_vals[:, 0].reshape(X.shape)
mo2 = mo_vals[:, 1].reshape(X.shape)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
im0 = axs[0].contourf(X, Z, mo1, levels=30, cmap='RdBu')
#axs[0].contour(X, Z, mo1, levels=12, colors='k', linewidths=0.4)
axs[0].set_title("MO 1 (bonding)")
axs[0].set_xlabel("x (bohr)"); axs[0].set_ylabel("z (bohr)")
fig.colorbar(im0, ax=axs[0])

im1 = axs[1].contourf(X, Z, mo2, levels=30, cmap='RdBu')
#axs[1].contour(X, Z, mo2, levels=12, colors='k', linewidths=0.4)
axs[1].set_title("MO 2 (antibonding)")
axs[1].set_xlabel("x (bohr)"); axs[1].set_ylabel("z (bohr)")
fig.colorbar(im1, ax=axs[1])

plt.tight_layout()
plt.show()


In [ ]:
# @title Static H2 orbitals at min & max distances + HF/FCI vs distance (Plotly) { display-mode: "form" }
basis_set = "STO-3G"   # @param ["STO-3G", "4-31G", "6-31G*", "6-311G**", "cc-pVDZ", "cc-pVTZ"]
dmin = 0.6             # @param {type:"number"}  # bohr
dmax = 3.0             # @param {type:"number"}  # bohr
nsteps_energy = 25     # @param {type:"integer"} # samples for energy curves
grid_n = 160           # @param {type:"integer"} # points per axis (e.g., 96–200)
slice_axis = "y"       # @param ["x","y","z"]
slice_value = 0.0      # @param {type:"number"}  # plane position (bohr)


# --- build H2 molecule with bond along x-axis ---
def make_mol(R, basis):
    atom = f"H {-R/2} 0 0; H {R/2} 0 0"
    mol = gto.Mole()
    mol.atom = atom
    mol.unit = 'Bohr'
    mol.basis = basis
    mol.spin = 0
    mol.charge = 0
    mol.build()
    return mol

# --- slice grid ---
L = float(dmax/2+2.)
N = int(grid_n)
x = np.linspace(-L, L, N)
y = np.linspace(-L, L, N)
z = np.linspace(-L, L, N)

if slice_axis.lower() == "y":
    # plane y = slice_value → show (x,z)
    X, Z = np.meshgrid(x, z, indexing="xy")
    coords = np.column_stack([X.ravel(), np.full(X.size, slice_value), Z.ravel()])
    ax_x, ax_y = x, z
    xlabel, ylabel = "x (bohr)", "z (bohr)"
    def nuclei_xy(R): return np.array([-R/2, +R/2]), np.array([0.0, 0.0])  # along x

elif slice_axis.lower() == "x":
    # plane x = slice_value → show (y,z)
    Y, Z = np.meshgrid(y, z, indexing="xy")
    coords = np.column_stack([np.full(Y.size, slice_value), Y.ravel(), Z.ravel()])
    ax_x, ax_y = y, z
    xlabel, ylabel = "y (bohr)", "z (bohr)"
    def nuclei_xy(R): return np.array([0.0, 0.0]), np.array([0.0, 0.0])   # both project to origin

else:  # z-slice
    X, Y = np.meshgrid(x, y, indexing="xy")
    coords = np.column_stack([X.ravel(), Y.ravel(), np.full(X.size, slice_value)])
    ax_x, ax_y = x, y
    xlabel, ylabel = "x (bohr)", "y (bohr)"
    def nuclei_xy(R): return np.array([-R/2, +R/2]), np.array([0.0, 0.0])  # along x

# --- compute slices & energies ---
def compute_slices_and_energies(R, basis):
    mol = make_mol(R, basis)
    mf = scf.RHF(mol)
    mf.verbose = 0 
    mf.kernel()
    nmo = mf.mo_coeff.shape[1]; occ = mol.nelectron // 2
    homo_idx, lumo_idx = occ - 1, min(occ, nmo-1)
    ao_vals = mol.eval_gto('GTOval_sph', coords)
    mo_vals = ao_vals @ mf.mo_coeff
    homo_slice = mo_vals[:, homo_idx].reshape(N, N)
    lumo_slice = mo_vals[:, lumo_idx].reshape(N, N)
    E_HF = float(mf.e_tot)
    h1 = mf.mo_coeff.T @ mf.get_hcore() @ mf.mo_coeff
    eri = ao2mo.restore(1, ao2mo.kernel(mol, mf.mo_coeff), nmo)
    cis = fci.FCI(mol, mf.mo_coeff)
    E_FCI, _ = cis.kernel(h1, eri, nmo, mol.nelectron)
    return homo_slice, lumo_slice, homo_idx, lumo_idx, E_HF, E_FCI

# --- evaluate at dmin and dmax ---
Hs_min, Ls_min, hidx_min, lidx_min, Ehf_min, Efci_min = compute_slices_and_energies(dmin, basis_set)
Hs_max, Ls_max, hidx_max, lidx_max, Ehf_max, Efci_max = compute_slices_and_energies(dmax, basis_set)

# color scale
Zmax = max(abs(Hs_min).max(), abs(Ls_min).max(), abs(Hs_max).max(), abs(Ls_max).max())
cmin, cmax = -Zmax, Zmax

# energy curves
distances = np.linspace(dmin, dmax, nsteps_energy)
E_HF_curve, E_FCI_curve = [], []
for R in distances:
    _, _, _, _, Ehf, Efci = compute_slices_and_energies(R, basis_set)
    E_HF_curve.append(Ehf); E_FCI_curve.append(Efci)

# --- plot ---
# Make square-ish orbital cells and a shorter energy row
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{"type":"heatmap"}, {"type":"heatmap"}],
           [{"type":"heatmap"}, {"type":"heatmap"}],
           [{"type":"scatter", "colspan":2}, None]],
    row_heights=[0.3, 0.3, 0.4],     # ↑ taller top rows, smaller bottom row
    column_widths=[0.5, 0.5],
    horizontal_spacing=0.24,
    vertical_spacing=0.1,
    subplot_titles=(
        f"HOMO (MO {hidx_min}) at R={dmin:.2f} bohr",
        f"LUMO (MO {lidx_min}) at R={dmin:.2f} bohr",
        f"HOMO (MO {hidx_max}) at R={dmax:.2f} bohr",
        f"LUMO (MO {lidx_max}) at R={dmax:.2f} bohr",
        "HF & FCI energies vs distance"
    ),
)

def add_panel(Zvals, row, col, R):
    fig.add_trace(go.Heatmap(x=ax_x, y=ax_y, z=Zvals,
                             coloraxis="coloraxis", showscale=False, showlegend=False), row=row, col=col)
    fig.add_trace(go.Contour(x=ax_x, y=ax_y, z=Zvals,
                             contours=dict(start=cmin, end=cmax, size=(cmax-cmin)/24, coloring='none'),
                             line=dict(width=1, color='rgba(100,100,100,0.4)'), showscale=False, showlegend=False), row=row, col=col)
    fig.add_trace(go.Contour(x=ax_x, y=ax_y, z=Zvals,
                             contours=dict(start=0, end=0, size=1, coloring='none'),
                             line=dict(width=3, color='black'), showscale=False, showlegend=False), row=row, col=col)
    nx, ny = nuclei_xy(R)
    fig.add_trace(go.Scatter(x=nx, y=ny, mode="markers",
                             marker=dict(size=10, color='black', symbol="circle"),
                             showlegend=False), row=row, col=col)

# four orbital panels
add_panel(Hs_min, 1, 1, dmin); add_panel(Ls_min, 1, 2, dmin)
add_panel(Hs_max, 2, 1, dmax); add_panel(Ls_max, 2, 2, dmax)

# bottom energy panel
fig.add_trace(go.Scatter(x=distances, y=E_HF_curve, mode="lines+markers", name="HF"), row=3, col=1)
fig.add_trace(go.Scatter(x=distances, y=E_FCI_curve, mode="lines+markers", name="FCI"), row=3, col=1)

# Make the overall figure taller so those two rows can be square
fig.update_layout(title=f"H₂ MOs — {basis_set} — bond along x-axis — slice {slice_axis}={slice_value} bohr",
width=1200*0.75, height=1400*0.75, margin=dict(l=40, r=120, t=90, b=40), legend=dict(orientation="h", x=0.5, y=0.3, xanchor="center"))

# Lock equal scale AND fix visible ranges so all four panels match
for (r, c) in [(1,1),(1,2),(2,1),(2,2)]:
    fig.update_xaxes(range=[-L, L], row=r, col=c)
    fig.update_yaxes(range=[-L, L], row=r, col=c)

# Keep the 1:1 aspect per panel (so chemistry looks right)
fig.update_yaxes(scaleanchor="x1", row=1, col=1)
fig.update_yaxes(scaleanchor="x2", row=1, col=2)
fig.update_yaxes(scaleanchor="x3", row=2, col=1)
fig.update_yaxes(scaleanchor="x4", row=2, col=2)

# labels
for r in (1,2):
    for c in (1,2):
        fig.update_xaxes(title_text=xlabel, row=r, col=c)
        fig.update_yaxes(title_text=ylabel, row=r, col=c)
fig.update_xaxes(title_text="distance (bohr)", row=3, col=1)
fig.update_yaxes(title_text="Energy (Hartree)", row=3, col=1)

fig.show()

**Homework Assignment**

Pick one (or more) of the following projects:
1. Modify the code to xxx
2. Modify the code to xxx
3. Modify the code to handle one of the following systems:
    * xxx
    * xxx 
    * xxx

For the modified code, answer the following questions:

NOTE: It is not necessary that the modified code produces an animation, but you should be able to visualize the results of the simulation in some way. 